# Exploring the data, basic fitting and visualization

Now that we've learned the basics of ridge regression, lets use it to build an encoding model with calcium imaging data. We will make use of scikit-learn, a popular machine learning library, to handle the regression fitting and crossvalidation.

#### Handle the imports

In [ ]:
#First import the standard toolboxes
import numpy as np 
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
from tqdm import tqdm

#Import the specific functions for this class
import encoding_tools

#### Download the data from the gdrive and load it

In [ ]:
data_file = encoding_tools.download_neural_data("miniscope") #Download function from io
data_dict = np.load(data_file, allow_pickle = True).tolist()

#Create a variable for each keyword in the dictionary
print('\nVariables:\n')
for key,val in data_dict.items():
        exec(key + '=val')
        print(key)


#### What are these data?

The data we have loaded were obtained from a mouse performing a two-alternative forced choice (2AFC) task. Mice must discriminate between high-frequency (> 12 Hz) and low-frequency (< 12 Hz) click trains. Depending on wether the stimulus was low or high rate, the mouse must poke its nose into the left or right spout to recieve a water reward. Neurons in the anterior cingulate cortex were recorded with a head-mounted miniscope during the task. Animal movements were also tracked with high resolution cameras.

 What do the indiviudal variables mean?

<ul>
    <li> <strong>desing_matrix</strong>: This is the heart of the encoding model we are going to build. It contains a set of task and movement variables aligned to four distinct phases of the task, size is [number of total frames, regressor number]. It is equivalent to the matrix X from the previous notebook.</li>
 <li><strong>Y_raw_fluorescence</strong>: The fluorescence traces from all the neurons recorded algined to the same task phases as the task- and movement variables, size is [number of total frames, neuron number]</li>
 <li><strong>neuron_footprints</strong>: The spatial filters for the recorded neurons, size is [300 pixels, 300 pixels, neuron number]. </li>
 <li><strong>timepoints_per_trial</strong>: The number of frames per trial.</li>
  <li><strong>frame_rate</strong>: The acquisition frame rate for the miniscope data.</li>
  <li><strong>aligned_segment_start</strong>: Marks the frames where the activity was aligned to a new task event. These also mark different task phases: Timepoints 0 to aligned_segment_start[0] contains the 1 second prior to trial initiation, the next phase is the 1 second from stimulus onset followed by action phase spanning from 200 ms before to 300 ms after movement onset and finally the outcome phase with 2 seconds after outcome presentation. The reason why the trial is split up into these phases is that there are variable delays before and after stimulus presentation and the choice report also happens at different speed.</li>
 </ul>


## Exercise 1: What is the structure of the design matrix?

Lets start by looking at the task variables inside the design matrix:

<ul>
    <li>time (somewhat like an intercept term, explains fluctuations that happen every trial)</li>
    <li>choice (left = 0, right = 1)</li>
    <li>stimulus strength</li>
    <li>outcome (incorrect = 0, correct = 1)</li>
    <li>previous choice  (left = 0, right = 1)</li>
    <li>previous outcome (incorrect = 0, correct = 1)</li>
</ul> 

Let's  print some of the keys of the data frame and plot the task variable columns!

In [ ]:
#Print some of the keys of the design matrix
print(design_matrix.keys()[np.arange(0,6 * timepoints_per_trial, timepoints_per_trial)])
print(f'Design matrix shape is {np.array(design_matrix).shape}')

#Plot the task variables
N_TRIALS_TO_PLOT = 29 # starting from first trial
N_REGRESSORS_TO_PLOT = 6 # starting from first regressor

plt.figure()
plt.imshow(np.array(design_matrix)[:N_TRIALS_TO_PLOT * timepoints_per_trial,:N_REGRESSORS_TO_PLOT * timepoints_per_trial], aspect='auto', cmap='seismic')
#use np.array() on the design_matrix to be able to index numerically instead of by key
for k in range(N_REGRESSORS_TO_PLOT):
    plt.axvline(k * timepoints_per_trial, color='k', linestyle='--', linewidth=0.5)
#for k in range(N_TRIALS_TO_PLOT):
#    plt.axhline(k * timepoints_per_trial, color='k', linestyle='--', linewidth=0.5, alpha=.4)
    
plt.colorbar()
plt.xlabel('Regressor')
plt.ylabel('Time (frames)')
plt.show()

######---Try answering the following questions:
# Does the t0 on the keys mean?
# Why do we need many columns to represent one variable (such as choice)?
# Why is the first column within the dashed lines full 1's on the diagonal?
# Why are the other ones not all diagonals?
# What do negative values along the diagonal indicate?

We have now generated a plot of our design matrix (X). Now let's take a look at our neural data (Y), and then we will come back to modeling the relationship between X and Y. 

## Exercise 2: How do task variables affect the activity of ACC neurons?

The aim of this exercise is to plot peri-event time histograms (PETH's) for two neurons for different choices and outcomes. This will act as an important sanity check for our encoding model! (If the encoding model finds a neuron that strongly encodes choice, then the PETH's for leftward vs rightward choice should be quite different). 

<ol>
    <li> Extract the choice and outcomes for each trial from the design matrix. </li>
    <li> Reshape the response matrix, Y_raw_fluorescence, to be a 3-dimensional array refelcting time, neuron id and trial number. </li>
    <li> Complete the missing lines of code in the function that calculates and plots the mean +/- sem trace for a respective binary value of a task variable </li>
</ol>

In the following we will plot PETH's for <strong>neuron 56</strong> and <strong>neuron 12</strong>. Let's first see how these cells look in the field of view.

In [ ]:
#Show the two example neurons: neuron 56 in red and neuron 12 in cyan superimposed on top of all the other identified cells (gray scale)
encoding_tools.overlay_neurons(neuron_footprints, 56, 12, 12) #Pass the neuron ids for the red, green and blue channel

In [ ]:
# 1. Extract choice and previous_choice for each trial from the design matrix: Use what you have learned about the
#    structure of the design matrix to generate a vector of choices and previous_choices, one entry per trial.

n_total_frames = design_matrix.shape[0] #total number of frames in the session

choice = #TODO: get choices for each trial from design matrix 
previous_choice = #TODO: get previous choices for each trial from design matrix

plt.plot(choice)
plt.ylabel('Choice')
plt.xlabel('Trial number')

We now have arrays that define the choice and the previous choice for the given trials, we now need to reshape the neural data from [n_total_frames, n_neurons] to [frames_per_trial, n_neurons, n_trials]

In [ ]:
# Reshape the flourescence data matrix to obtain a timepoints x neuron id x trial matrix.

n_neurons = Y_raw_fluorescence.shape[1]
n_trials = int(design_matrix.shape[0] / timepoints_per_trial)

Y_3d = #TODO: reshape Y_raw_flourescence to the following shape [frames_per_trial, n_neurons, n_trials]

print(f'The shape of the calcium signal matrix was {Y_raw_fluorescence.shape}')
print(f'The shape of the calcium signal matrix is now {Y_3d.shape}')

In [ ]:
# 3. Please complete the function below that can be used to calculate and plot mean +/- sem traces for a specified neuron
#    and a specific binary task variable.

def plot_mean_trace(Y_3d,
                    task_var, #the binary array of task variables for each trial (stimulus, choice, etc.)
                    task_var_name, #used for figure title
                    neuron_id, 
                    aligned_segment_start, 
                    ax,
                    colors=['#062999','#b20707'], 
                    spacer=7, 
                    var_value_strings=['Left', 'Right']):

    alignment_position_x_vect = [-1, 0, 6, 0] #These are the indices of the x_vect element where the alignment happend. 
    
    average_trace = np.stack((np.mean(Y_3d[:, neuron_id, task_var==0], axis=1),
                              np.mean(Y_3d[:, neuron_id, task_var==1], axis=1))).T # stack the different conditions along a new dimension
    
    sem_trace = #TODO: calculate the SEM for the different task_var values and stack them along a new dimension, as above.
    
    for n in range(average_trace.shape[1]): #Loop through the conditions (left and right in the case of choice)
        for k in range(len(aligned_segment_start)): #Loop through the different task segments
            
            if k < len(aligned_segment_start) - 1 : #Split by different segments
                plot_idx = np.arange(aligned_segment_start[k], aligned_segment_start[k+1]) #Extract indices of the current segment
                x_vect = (plot_idx + k*spacer) / frame_rate #Create a time vector to plot the data to
                line_label = None 
            else:
                plot_idx = np.arange(aligned_segment_start[k], Y_3d.shape[0])
                x_vect = (plot_idx + k*spacer) / frame_rate
                line_label = var_value_strings[n] #only use a label on the last segment

            # TODO: Plot the sem and the average traces of the respective segment agains the time vector
            #       to the axes ax using fill_between and plot methods. Remember that we are not only looping
            #       through aligned segments but also through task variable conditions (True or False).
            #       As keyword arguments for fill_between use: color = colors[n], alpha = 0.5, linewidth = 0
            #       As keyword aruments to plot use: color = colors[n], label = line_label

            ax.fill_between() #TODO: plot sem here
            ax.plot() #TODO: plot mean here
    

            if n == 0: #Plot vertical alignment timepoints only once in the loop
                ax.axvline(x_vect[alignment_position_x_vect[k]], color = 'k', linewidth = 0.8, linestyle = '--')
            
    #Do some figure formating
    ax.set_xlabel('Time (s)')
    ax.set_ylabel('Fluorescence (A.U.)')
    ax.set_title(f'{task_var_name} for neuron {neuron_id}')
    ax.legend()
    ax.set_ylim([-5, 40]) #Pre-set for this data
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.spines['left'].set_bounds([ax.get_yticks()[0], ax.get_yticks()[-1]]) 
    ax.spines['bottom'].set_bounds([ax.get_xticks()[1], ax.get_xticks()[-2]]) #In this case the plotting is bounded by invisible ticks, thus 1 and -2
    return


#Now lets call our function and plot the traces
#First create a figure with four subplots
axes_list = encoding_tools.create_subplot_axes(2, 2, fig_size=(10,10)) 

#Now we can fill the subplots
plot_mean_trace(Y_3d, choice, 'Choice', 56, aligned_segment_start, axes_list[0])
plot_mean_trace(Y_3d, choice, 'Choice', 12, aligned_segment_start, axes_list[1])
plot_mean_trace(Y_3d, previous_choice, 'Previous choice', 56, aligned_segment_start, axes_list[2], colors = ['#4fa97a', '#ac3cbd'])
plot_mean_trace(Y_3d, previous_choice, 'Previous choice', 12, aligned_segment_start, axes_list[3], colors = ['#4fa97a', '#ac3cbd'])

print('Compare the PETHs for the two neurons. What do you think each neuron is responding to?\nAt which moment in time are the task variables represented differentially?')

#### Prepare for model fitting

Here, we will set the inputs and outouts up for the following hyperparameter optimization and the model fitting and performance evaluation. We will need to z-score the response matrix, Y_raw_fluorescence, and the design_matrix as well. The latter one is extremely important to make sure that the different regressors are measured on the same scale, so that weights are more comparable and have similar impacts on the ridge parameter selection. However, here, we choose not to z-score 1) the sin and cos of the head orientation angles because independent z-scoring would abolish their relation and thus remove angle information 2) the binary task variables because leaving them in their binary form preserves the interpretability of the model. <br>
<strong>Can you imagine why this is?<strong/>

In [ ]:
#z-score the response matrix
from scipy.stats import zscore
Y = zscore(Y_raw_fluorescence, axis=0)

#z-score the respective column of the design matrix
scaler_obj = encoding_tools.standardize_x_cols(column_idx=np.arange(832, 1258)).fit(design_matrix) #Columns 0:827 are task vars, cols 828:832 are head orientation
X = scaler_obj.transform(design_matrix)
#Note: this class is defined in utils. It contains a fit and transform method, similar to StandardScaler from sklearn.preprocessing
#We'll get back to this below

## Exercise 3: Estimating the reularization strength of the model

As you have seen in the teaching modules and notebook No 1: <strong>1_Ridge_Regression</strong> the regularization strength (we use sklearn convention and call it alpha here) controls the shrinkage of the regressor weights and is critical to prevent overfitting. In this notebook we will use the sklearn class RidgeCV. RidgeCV searches through a grid of provided candidate alphas and determines the best alpha value using a built-in leave-one-out cross validation. <br>
In the following exercise you will be asked to:
<ol>
    <li> Choose appropriate parameters <strong>fit_intercept</strong> and <strong>alphas_per_target</strong> for RidgeCV.</li>
    <li> Perform the grid search.</li>
    <li> Retrieve the found regularization strengths as <strong>alphas</strong>.</li>
</ol>

In [ ]:
print(X.shape)
print(Y.shape)

In [ ]:
test_range = 10**np.linspace(-10,9,50) #Get search values for alpha on log scale

# 1. Define the RidgeCV paramters fit_intercept and alphas_per_target and explain why you chose them

fit_intercept = #TODO: set this boolean to the proper value
alpha_per_target = #TODO: set this boolean to the proper value

# 2. Perform the grid search on the provided arrays X and Y

from sklearn.linear_model import RidgeCV

ridge_grid_search = #TODO: create the RidgeCV() object here with the proper arguments, including the ones you specified above

ridge_grid_search.fit(X,Y) #perform the alpha grid search

alphas = #TODO: get the best alpha values after fitting. There will be an alpha value for each neuron

In [ ]:
#Plot histogram of the best alpha values for the all of the neurons
fig,ax = plt.subplots()
ax.hist(alphas, bins=test_range)
ax.set_xscale('log')
ax.set_xlabel('Best alpha value')
ax.set_ylabel('Count')

## Exercise 4: Assemble a pipeline for data preprocessing and model fitting with the found regularization strengths, alpha

For this exercise you will implement a Pipeline to fit ridge models for the 10 splits of the data and evaluate the performance of each of them. 
<ol>
    <li> Use sklearns Pipeline object to create a model that will standardize the data within each fold before fitting the ridge model. It is best practice to standardize within folds.
    <li> Use the pipeline to fit the model on the training data with the output <strong>fits</strong> and predict the testing data, retrieving <strong>predictions</strong>.</li>

In [ ]:
#Start by setting up the parameters of the model
from sklearn.model_selection import KFold
from sklearn.linear_model import Ridge
from sklearn.pipeline import Pipeline
from sklearn.metrics import r2_score

N_FOLDS = 10
X = np.array(design_matrix) #convert to array 

custom_scaler = encoding_tools.standardize_x_cols(column_idx = np.arange(832, 1258)) #step 1 of the Pipeline
ridge_model = Ridge(alpha=alphas, fit_intercept=fit_intercept) #step 2 of the pipeline

# 1. Create the pipeline for the fitting below. The pipeline should contain a 'scaler' step that uses the custom_scaler from above
#    and a 'ridge' step that uses ridge_model from above. Pipelines are nice because now the data standardization happens within each fold, which is best practice.

pipe = #TODO: create the Pipeline here. You may want to refer to the sklearn documentation.

betas, r_squared = [], [] # we will store weights and r2 for each fold here
kf = KFold(n_splits=N_FOLDS, shuffle=True) 

for train_index, test_index in kf.split(X, Y): #iterate over cv-folds.
    
    # 2. Fit the training data sets using the pipleline and retrieve the result as fits. Then predict the unseen testing data
    #    with the pipeline and store the results as predictions

    fits = #TODO: fit the Pipeline object with TRAINING data. Use the .fit() method
    predictions = #TODO: generate predictions from the Pipeline object with TESTING data. Use the .predict() method

    r_squared.append(r2_score(Y[test_index,:], predictions, multioutput='raw_values'))
    betas.append(pipe[-1].coef_)
    
betas = np.squeeze(betas)
r_squared = np.squeeze(r_squared)

#Plot the sorted r_squared values
average_r2 = np.mean(r_squared, axis=0)
idx = np.argsort(average_r2)[::-1]
example_neuron_idx = [np.where(idx==56)[0][0], np.where(idx==12)[0][0]]

ax = plt.figure().add_subplot(111)
ax.plot(average_r2[idx], color = 'k')
ax.scatter(example_neuron_idx, average_r2[[56,12]], color = 'g')
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_bounds([0,0.8])
ax.spines['bottom'].set_bounds([0,550])
ax.set_xlabel('Neuron number')
ax.set_ylabel('cross-validated R\u00b2') 
ax.text(70, 0.66, 'Neuron 56 and neuron 12')


## Exercise 5: Plot the choice and previous choice encoding over the trial duration as estimated by the encoding model

The aim of the final exercise of this notebook is to visualize the estimated weights for the previous choice and choice regressors across trial time.
<ol>
    <li> Average the beta weights of the different fold to generate the mean weights as <strong>average_beta</strong>.</li>
    <li> Using what you've learned about the structure of the design matrix retrieve the beta weights for choice and previous choice over the trial duration as <strong>choice_betas</strong> and <strong>previous_choice_betas</strong>.</li>
    <li> Compare the compare the obtained plots and also look again at the plots from exercise 2. Discuss the questions printed above the plots.</li>
</ol>

In [ ]:
# 1. Start by getting the average beta weights

average_beta = #TODO: get the average beta weights across the 10 folds

# 2. Retrieve the betas for time, choice, and previous choice and name them choice_betas and previous_choice_betas. You may
#    need to refer back to the beginning of this notebook to see what indices those came from. Make sure
#    to build these varaibles so that the trial times are rows and the columns are neurons.

time_betas = #TODO: get the betas for time
choice_betas = #TODO: get the betas for choice
previous_choice_betas = #TODO: get the betas for previous choice

In [ ]:
def plot_task_var_betas(task_var_betas, time_betas, task_var_name, neuron_id, aligned_segment_start, ax, colors = ['#062999','#b20707'], spacer = 7, var_value_strings = ['Left', 'Right']):
    
    beta_traces = np.stack((time_betas[:,neuron_id], task_var_betas[:,neuron_id]),axis=1)
    alignment_position_x_vect = [-1, 0, 6, 0] #These are the indices of the x_vect element where the alignment happend. 
    
    for n in range(beta_traces.shape[1]): #Loop through the average traces
        for k in range(len(aligned_segment_start)): #Loop through the different task segments
            
            if k < len(aligned_segment_start) - 1 : #Split by different segments
                plot_idx = np.arange(aligned_segment_start[k], aligned_segment_start[k+1]) #Extract indices of the current segment
                x_vect = (plot_idx + k*spacer) / frame_rate #Create a time vector to plot the data to
                line_label = None 
            else:
                plot_idx = np.arange(aligned_segment_start[k], beta_traces.shape[0])
                x_vect = (plot_idx + k*spacer) /frame_rate
                line_label = var_value_strings[n] #only use a label on the last segment
            ax.plot(x_vect, beta_traces[plot_idx,n], color = colors[n], label = line_label)
            
            if n == 0: #Plot vertical alignment timepoints only once in the loop
                ax.axvline(x_vect[alignment_position_x_vect[k]], color = 'k', linewidth = 0.8, linestyle = '--')
            
    
    #Do some figure formating
    ax.set_xlabel('Time (s)')
    ax.set_ylabel('Regression weight')
    ax.set_title(f'{task_var_name} for neuron {neuron_id}')
    ax.legend()
    ax.set_ylim([-2, 2.2]) #Pre-set for this data
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.spines['left'].set_bounds([ax.get_yticks()[0], ax.get_yticks()[-2]]) 
    ax.spines['bottom'].set_bounds([ax.get_xticks()[1], ax.get_xticks()[-2]])
    return

#Now lets call our function and plot the traces!

#First create a figure with four subplots
axes_list = encoding_tools.create_subplot_axes(2, 2) #From visualizations

#Now we can fill the subplots
plot_task_var_betas(choice_betas, time_betas, 'Choice', 56, aligned_segment_start, axes_list[0], colors = ['#8b8b8b', '#b20707'], var_value_strings = ['Intercept', 'Right choice'])
plot_task_var_betas(choice_betas, time_betas, 'Choice', 12, aligned_segment_start, axes_list[1], colors = ['#8b8b8b', '#b20707'], var_value_strings = ['Intercept', 'Right choice'])
plot_task_var_betas(previous_choice_betas, time_betas, 'Previous choice', 56, aligned_segment_start, axes_list[2], colors = ['#8b8b8b', '#ac3cbd'], var_value_strings = ['Intercept', 'Previous right choice'])
plot_task_var_betas(previous_choice_betas, time_betas, 'Previous choice', 12, aligned_segment_start, axes_list[3], colors = ['#8b8b8b', '#ac3cbd'], var_value_strings = ['Intercept', 'Previous right choice'])

# 3. Discuss these questions:
print('Why are the lines labeled Intercept and Right choice / Previous right choice on these plots?')
print('What is the interpretation of these lines and how would you reconstruct a PETH for a right choice trial from the regressor weights?')
print('Do these neurons mix encoding of different task variables?')
print('How are these plots different from the ones in exercise 2? Do the plots below suggest different task variable encoding?')
      

## This is it for this notebook!

Many thanks to Adi Nair for putting together the DataSAI summer school program, Max Melin and Gabriel Rojas-Bowe for putting together the content of this class and Anne Churchland and the Churchland lab for their support and inputs.
